<a href="https://colab.research.google.com/github/alan-italo/Estudos/blob/master/AnalisePortalTransparencia_Alan_Almeida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Configurando o Spark no python

In [1]:
# Instalação e config. do java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#Download do apache spark
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

#Descompactando o apache spark
!tar xf spark-3.0.1-bin-hadoop3.2.tgz

#Instalacao do findspark
!pip install -q findspark

2 - Configurando o ambiente JAVA

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

3 - Configurando a sessao do Spark

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

4 - Validando a configuracao da versao do spark

In [4]:
import pyspark
print(pyspark.__version__)

3.0.1


5. consultando dados usando CURL
Após acessar o site do governo e gerar a rash de acesso 

In [5]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: c0b9a7962c523e4d756e81a406c35fd6' 'http://www.portaltransparencia.gov.br/api-de-dados/cartoes?mesExtratoInicio=08%2F2020&mesExtratoFim=09%2F2020&pagina=1'> consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26369    0 26369    0     0   8996      0 --:--:--  0:00:02 --:--:--  8993


In [8]:
sc = spark.sparkContext
path = "/content/consulta_portal_transparencia.json" 
df = spark.read.json(path)

In [9]:
df.printSchema()
df.show()

root
 |-- dataTransacao: string (nullable = true)
 |-- estabelecimento: struct (nullable = true)
 |    |-- cnae: struct (nullable = true)
 |    |    |-- classe: string (nullable = true)
 |    |    |-- codigoClasse: string (nullable = true)
 |    |    |-- codigoDivisao: string (nullable = true)
 |    |    |-- codigoGrupo: string (nullable = true)
 |    |    |-- codigoSecao: string (nullable = true)
 |    |    |-- codigoSubclasse: string (nullable = true)
 |    |    |-- divisao: string (nullable = true)
 |    |    |-- grupo: string (nullable = true)
 |    |    |-- secao: string (nullable = true)
 |    |    |-- subclasse: string (nullable = true)
 |    |-- codigoFormatado: string (nullable = true)
 |    |-- complementoEndereco: string (nullable = true)
 |    |-- dataAbertura: string (nullable = true)
 |    |-- descricaoLogradouro: string (nullable = true)
 |    |-- enderecoEletronico: string (nullable = true)
 |    |-- localidadePessoa: string (nullable = true)
 |    |-- municipio: struct

6. Criando uma tabela para analise

In [10]:
dfCompleted = df.selectExpr("dataTransacao","estabelecimento.razaoSocialReceita","estabelecimento.nome","estabelecimento.municipio.uf.sigla","estabelecimento.tipoPessoa", "portador.nome as nome_do_portador","tipoCartao.descricao","valorTransacao")
dfCompleted.show()

+-------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------+
|dataTransacao|  razaoSocialReceita|                nome|sigla|          tipoPessoa|    nome_do_portador|           descricao|valorTransacao|
+-------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------+
|   19/08/2020|M & P COMERCIO DE...|M & P COMERCIO DE...|   BA|Entidades Empresa...|ANTONIO VILAS BOA...|Cartão de Pagamen...|        260,00|
|   06/07/2020|LEMOS LABORATORIO...|LEMOS LABORATORIO...|   MG|Entidades Empresa...|LIVIA FRANCO PERE...|Cartão de Pagamen...|      1.400,00|
|   16/07/2020|      SEM INFORMACAO|      SEM INFORMACAO|   -1|      Sem Informação|BRUNO DOS SANTOS ...|Cartão de Pagamen...|      1.242,00|
|   20/08/2020|       NAO SE APLICA|       NAO SE APLICA|   -1|      Sem Informação|ASCLE BRITO DE SOUZA|Cartão de Pagamen...|        180,00|
|   20

In [11]:
dfCompleted.createOrReplaceTempView("dfCompleted")

7. Criando uma analise de pessoas por estado

In [13]:
monthTravels = spark.sql("select dataTransacao as data, nome_do_portador as nome, sigla as estado, valorTransacao as valor from dfCompleted where sigla like '%MG%' ")
monthTravels.show()

+----------+--------------------+------+--------+
|      data|                nome|estado|   valor|
+----------+--------------------+------+--------+
|06/07/2020|LIVIA FRANCO PERE...|    MG|1.400,00|
+----------+--------------------+------+--------+

